# Assigning Regions to Sale Representatives at Pfizer Turkey 


In [17]:
import pandas as pd
import numpy as np

assignment = pd.read_csv("assignment.csv")
distances = pd.read_csv("brick_rp_distances.csv")
workload = pd.read_csv("bricks_index_values.csv")

In [15]:
print(assignment.columns)
workload["index_value"].sum()
bricks = assignment["Bricks_Assigned"]
bricks

Index(['SR#', 'Center_Brick#', 'Bricks_Assigned'], dtype='object')


0           [4,5,6,7,15]
1       [10,11,12,13,14]
2           [9,16,17,18]
3    [1,2,3,19,20,21,22]
Name: Bricks_Assigned, dtype: object